In [ ]:
%matplotlib inline

# Convert nodal coordinates field to local coordinate system {#ref_nodes_in_local_coordinate_system}

GCS: Global Coordinate System LCS: Local Coordinate System Currently,
there is no native operator to get nodal coordinates in an LCS. The
operator :class: [rotate
\<ansys.dpf.core.operators.geo.rotate.rotate\>]{.title-ref} rotates the
input field in GCS as per the input rotation matrix. So, if the LCS is
at the same origin as the GCS, only one operation using the
[rotate]{.title-ref} operator give the desired output. But for an LCS
with a transformed origin, after rotation a transformation along the
rotated position vector of LCS\'s origin is also needed to get the
correct coordinates in LCS.

The script below demonstrates the methodology using PyDPF.

Import necessary modules:


In [ ]:
from ansys.dpf import core as dpf
from ansys.dpf.core import examples

Create a model object to establish a connection with an example result
file:


In [ ]:
model = dpf.Model(examples.download_hemisphere())

Get the property [coordinates_field]{.title-ref} from :class: nodes
ansys.dpf.core.nodes\`


In [ ]:
ncoord_f = model.metadata.meshed_region.nodes.coordinates_field

Get the rotation matrix of the LCS ID 12 The first 9 values in the
[cs]{.title-ref} output is the rotation matrix


In [ ]:
cs = model.operator(r"mapdl::rst::CS")
cs.inputs.cs_id.connect(12)
cs_rot_mat = cs.outputs.field.get_data().data.T[0:9]

Create a 3x3 rotation matrix field - [rot_mat_f]{.title-ref}


In [ ]:
rot_mat_f = dpf.fields_factory.create_scalar_field(1)
rot_mat_f.data = cs_rot_mat

Create a 3D vector field for the position vector of the LCS\'s origin
and rotate the origin as per the rotation matrix of the LCS. The last 3
entries of [cs]{.title-ref} output is the LCS\'s origin in GCS.


In [ ]:
pos_vec = dpf.fields_factory.create_3d_vector_field(1)
pos_vec.data = cs.outputs.field.get_data().data.T[-3:]
pos_vec_rot = dpf.operators.geo.rotate(field=pos_vec,
                                       field_rotation_matrix=rot_mat_f)

Get rotated nodal coordinates field.


In [ ]:
ncoord_rot_f = dpf.operators.geo.rotate(field=ncoord_f,
                                        field_rotation_matrix=rot_mat_f)

Transform rotated nodal coordinates field along rotated position vector
-[pos_vec_rot]{.title-ref},


In [ ]:
pos_vec_rot_neg_f = dpf.operators.math.scale(field=pos_vec_rot,
                                             ponderation=-1.0)
pos_vec_rot_neg = pos_vec_rot_neg_f.outputs.field.get_data().data_as_list
ncoord_translate = dpf.operators.math.add_constant(field=ncoord_rot_f,
                                                   ponderation=pos_vec_rot_neg)

Get the nodal coordinates field [ncoord_lcs_f]{.title-ref} in LCS


In [ ]:
ncoord_lcs_f = ncoord_translate.outputs.field.get_data()

Coordinates of NID 1 in GCS


In [ ]:
print(ncoord_f.get_entity_data_by_id(1))

Coordinates of NID 1 in LCS


In [ ]:
print(ncoord_lcs_f.get_entity_data_by_id(1))